<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH15_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# RNNs
# Generally they can take on sequences of arbitrary length rather than fixed sized inputs

# Troubles of RNNs
# Unstable gradients - curable with recurrent dropout and recurrent layer normalization
# limited short term memory


# types of networks
# seq-to-seq (sequence produce a sequence later) like stock prices from N-1 days to today produce stock prices from N to tommorow
# seq-to-vec like words of a review to sentiment of said review (-1 or 1)
# vec-to-sec like image into sequence of labels that describe that image
# sec-to-vec (encoder) --> vec-to-sec (decoder) like a translator that takes a sequence of words from one language, encodes it into a "universal representation" in a vector and then decoders tries to decode it into a foreign language

# types of forecasting in time series
# prediction of future values (stonk)
# inputation of previous missing values ("postdiction")


In [ ]:
#common shit per usual
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
from pathlib import Path
np.set_printoptions(threshold=np.inf) # print pretty pony tables
np.set_printoptions(linewidth=2000)
# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
 # print("freq1: ", freq1, "\n freq2 ", freq2,  "\n offsets1", offsets1, "\n offsets2" ,offsets2)
  time = np.linspace(0, 1, n_steps)
  wave1 = 0.5 * np.sin((time - offsets1) * (freq1 * 30 + 10)) # amplitude offset (przesunięcie) frequency #okres
  wave2 = 0.5 * np.sin((time - offsets2) * (freq1 * 20 + 20)) 
  noise = 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
  # plt.plot(time, wave1[0])
  series = wave1 + wave2 + noise
  return series[..., np.newaxis].astype(np.float32)
  

In [ ]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]
#metrics
#naive prediction: just yeet the last known value of time series as your prediction
#after all, your stonks don't move that much day to day
#surprisingly hard to outperform

np.mean(keras.losses.mean_squared_error(y_valid, X_valid[:,-1]))
# about 0.07

0.07822794

In [ ]:
#metrics
#simple fully connected network and compare with that
#about 0.04

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[50,1]),
                                 keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=40)

Epoch 1/40
219/219 [==============================] - 1s 3ms/step - loss: 0.2307 - val_loss: 0.1139
Epoch 2/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0749 - val_loss: 0.0516
Epoch 3/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0413 - val_loss: 0.0349
Epoch 4/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0307 - val_loss: 0.0273
Epoch 5/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0246 - val_loss: 0.0219
Epoch 6/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0201 - val_loss: 0.0180
Epoch 7/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0167 - val_loss: 0.0151
Epoch 8/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0142 - val_loss: 0.0129
Epoch 9/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0123 - val_loss: 0.0113
Epoch 10/40
219/219 [==============================] - 1s 3ms/step - loss: 0.0110 - val_loss: 0.0101

In [ ]:
# your first RNN
model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(1, input_shape=[None,1])
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=40)

Epoch 1/40
219/219 [==============================] - 10s 47ms/step - loss: 0.1041 - val_loss: 0.0848
Epoch 2/40
219/219 [==============================] - 10s 47ms/step - loss: 0.0741 - val_loss: 0.0614
Epoch 3/40
219/219 [==============================] - 10s 45ms/step - loss: 0.0556 - val_loss: 0.0482
Epoch 4/40
219/219 [==============================] - 10s 45ms/step - loss: 0.0463 - val_loss: 0.0424
Epoch 5/40
219/219 [==============================] - 10s 46ms/step - loss: 0.0421 - val_loss: 0.0395
Epoch 6/40
219/219 [==============================] - 10s 47ms/step - loss: 0.0398 - val_loss: 0.0376
Epoch 7/40
219/219 [==============================] - 10s 47ms/step - loss: 0.0380 - val_loss: 0.0360
Epoch 8/40
219/219 [==============================] - 10s 46ms/step - loss: 0.0366 - val_loss: 0.0348
Epoch 9/40
219/219 [==============================] - 10s 45ms/step - loss: 0.0355 - val_loss: 0.0339
Epoch 10/40
219/219 [==============================] - 10s 46ms/step - loss: 0.034

In [ ]:
#deep RNN is just a vertical stack of RNNs
model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, batch_input_shape=[50, None,1]),
                                 keras.layers.SimpleRNN(20, return_sequences=True),
                                 keras.layers.SimpleRNN(1)
])
model.compile(loss="mse", optimizer="nadam")
for layer in model.layers:
    print(layer.output_shape)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=1, batch_size=50)

(50, None, 20)
(50, None, 20)
(50, 1)
140/140 [==============================] - 12s 88ms/step - loss: 0.0438 - val_loss: 0.0118


In [ ]:
for layer in model.layers:
    print(layer.output_shape)

In [ ]:
#deep RNN is just a vertical stack of RNNs
#deep RNN is just a vertical stack of RNNs
model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(1)
                                 
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)
for layer in model.layers:
    print(layer.output_shape)

Epoch 1/20
219/219 [==============================] - 21s 97ms/step - loss: 0.1375 - val_loss: 0.0120
Epoch 2/20
219/219 [==============================] - 21s 95ms/step - loss: 0.0081 - val_loss: 0.0068
Epoch 3/20
219/219 [==============================] - 21s 94ms/step - loss: 0.0054 - val_loss: 0.0054
Epoch 4/20
219/219 [==============================] - 20s 93ms/step - loss: 0.0044 - val_loss: 0.0043
Epoch 5/20
219/219 [==============================] - 20s 94ms/step - loss: 0.0038 - val_loss: 0.0040
Epoch 6/20
219/219 [==============================] - 20s 93ms/step - loss: 0.0035 - val_loss: 0.0037
Epoch 7/20
219/219 [==============================] - 20s 93ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 8/20
219/219 [==============================] - 21s 94ms/step - loss: 0.0033 - val_loss: 0.0034
Epoch 9/20
219/219 [==============================] - 21s 94ms/step - loss: 0.0032 - val_loss: 0.0035
Epoch 10/20
219/219 [==============================] - 21s 94ms/step - loss: 0.003

In [ ]:
#forecasting several steps ahead
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=2000)
series = generate_time_series(1, n_steps + 10)
print(series.shape)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
y0 = np.empty((1,1,1))
y1 = np.empty((1,1,1))
for step_ahead in range(10):
  y_pred = model.predict(X[:, step_ahead:])
  print(y_pred)
  y_pred_one = model.predict(X[:, step_ahead:])[..., np.newaxis] #dołóż nową oś by concatenate nie wyjebało w kosmos
  y0 = np.concatenate([y0, y_pred_one], axis = 1)
  y_pred_two = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
  y1 = np.concatenate([y1, y_pred_two], axis = 1)
  # tu się roluje, czyli bierze od kroku 0 do 50 i przewiduje 51, potem, gdy step_ahead=1 bierze X od kroku 1 do 51 (gdy się przypnie przewidywanie nr 51) i przewiduje 52 itd.
  X = np.concatenate([X, y_pred_one], axis = 1)
  

print(y0)
print(y1)

In [ ]:
z = np.empty((2,4))
print(z)
z = z[:, np.newaxis, :]
print(z)

In [ ]:
z = np.empty((2,4))
print(z)
z = z[:, :, np.newaxis]
print(z)

[[10.73607087  0.35755464  0.01855573  0.34476936]
 [ 0.60988677  0.73854846  0.66767079  0.4298614 ]]
[[[10.73607087]
  [ 0.35755464]
  [ 0.01855573]
  [ 0.34476936]]

 [[ 0.60988677]
  [ 0.73854846]
  [ 0.66767079]
  [ 0.4298614 ]]]


In [ ]:
z = np.empty((2,4))
print(z)
z = z[np.newaxis, :]
print(z)

[[10.73607087  0.35755464  0.01855573  0.34476936]
 [ 0.60988677  0.73854846  0.66767079  0.4298614 ]]
[[[10.73607087  0.35755464  0.01855573  0.34476936]
  [ 0.60988677  0.73854846  0.66767079  0.4298614 ]]]


[10.73607087 -0.35755464 -0.01855573  0.34476936  0.60988677  0.73854846  0.66767079  0.4298614   0.11216023 -0.19946051]


IndexError: ignored

In [ ]:
# instead of predicting one by one we can predict all at once
# performance is better
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]


model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(10)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)

X_new = series[:, :n_steps]
Y_pred = model.predict(X_new)


Epoch 1/20
219/219 [==============================] - 21s 97ms/step - loss: 0.1328 - val_loss: 0.0704
Epoch 2/20
219/219 [==============================] - 21s 96ms/step - loss: 0.0543 - val_loss: 0.0383
Epoch 3/20
219/219 [==============================] - 21s 96ms/step - loss: 0.0356 - val_loss: 0.0311
Epoch 4/20
219/219 [==============================] - 22s 99ms/step - loss: 0.0279 - val_loss: 0.0263
Epoch 5/20
219/219 [==============================] - 21s 98ms/step - loss: 0.0238 - val_loss: 0.0215
Epoch 6/20
219/219 [==============================] - 22s 100ms/step - loss: 0.0212 - val_loss: 0.0204
Epoch 7/20
219/219 [==============================] - 22s 100ms/step - loss: 0.0190 - val_loss: 0.0172
Epoch 8/20
219/219 [==============================] - 21s 96ms/step - loss: 0.0175 - val_loss: 0.0168
Epoch 9/20
219/219 [==============================] - 21s 98ms/step - loss: 0.0165 - val_loss: 0.0271
Epoch 10/20
219/219 [==============================] - 21s 98ms/step - loss: 0.0

In [ ]:
loss = keras.losses.MeanSquaredError(series[:, n_steps:], Y_pred[:, np.newaxis, :])
loss

In [ ]:
#instead of predicting 10 values at the last step we can predict 10 next values at every steo
#in other words from seqtovec to seqtoseq
#more gradients that will flow not only from teh end output throughtout time, but also from output of each time step
Y_pred.shape

In [ ]:
series[:, n_steps:].shape

In [ ]:
np.mean(keras.losses.mean_squared_error(series[:, n_steps:], Y_pred[:, np.newaxis]))

0.3919288

In [ ]:
series[:, n_steps:].shape

(10000, 10, 1)

In [ ]:
Y_pred.shape

(10000, 10)

In [ ]:
Y_pred.shape

(10000, 10)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
 92/219 [===========>..................] - ETA: 11s - loss: 0.2789

KeyboardInterrupt: ignored

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]


np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 21s 94ms/step - loss: 0.1295 - val_loss: 0.0651
Epoch 2/20
 72/219 [========>.....................] - ETA: 13s - loss: 0.0622

KeyboardInterrupt: ignored

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]


np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(10)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)

X_new = series[:, :n_steps]
Y_pred = model.predict(X_new)

Epoch 1/20
219/219 [==============================] - 21s 98ms/step - loss: 0.1326 - val_loss: 0.0649
Epoch 2/20
219/219 [==============================] - 21s 96ms/step - loss: 0.0518 - val_loss: 0.0402
Epoch 3/20
117/219 [===============>..............] - ETA: 9s - loss: 0.0398

KeyboardInterrupt: ignored

In [ ]:

model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(10)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)


Epoch 1/20
219/219 [==============================] - 21s 98ms/step - loss: 0.1131 - val_loss: 0.0592
Epoch 2/20
 61/219 [=======>......................] - ETA: 15s - loss: 0.0559

KeyboardInterrupt: ignored

In [ ]:
# ten steps ahead for all points 
# so having just t0 you predict t1-11, t0:1 - t2-12, ... , t0:50 - t51-60

In [ ]:
Y = np.empty((10000, n_steps, 10)) #10000 tables * 60 steps * 10 predictions for each step
for step_ahead in range(1, 10+ 1):
  Y[:, :, step_ahead - 1]

In [ ]:
series[:, 5:5+n_steps, 0]

In [ ]:
5+n_steps

Epoch 1/20
219/219 [==============================] - 22s 100ms/step - loss: 0.0859 - last_time_step_mse: 0.0645 - val_loss: 0.0501 - val_last_time_step_mse: 0.0280
Epoch 2/20
219/219 [==============================] - 21s 98ms/step - loss: 0.0423 - last_time_step_mse: 0.0204 - val_loss: 0.0363 - val_last_time_step_mse: 0.0166
Epoch 3/20
219/219 [==============================] - 21s 98ms/step - loss: 0.0343 - last_time_step_mse: 0.0152 - val_loss: 0.0317 - val_last_time_step_mse: 0.0146
Epoch 4/20
219/219 [==============================] - 21s 95ms/step - loss: 0.0319 - last_time_step_mse: 0.0151 - val_loss: 0.0283 - val_last_time_step_mse: 0.0118
Epoch 5/20
 23/219 [==>...........................] - ETA: 18s - loss: 0.0283 - last_time_step_mse: 0.0115

KeyboardInterrupt: ignored

In [ ]:
#few steps ahead at each and every point

np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))


Epoch 1/20
219/219 [==============================] - 20s 92ms/step - loss: 0.0806 - last_time_step_mse: 0.0609 - val_loss: 0.0476 - val_last_time_step_mse: 0.0279
Epoch 2/20
219/219 [==============================] - 20s 92ms/step - loss: 0.0420 - last_time_step_mse: 0.0221 - val_loss: 0.0365 - val_last_time_step_mse: 0.0181
Epoch 3/20
219/219 [==============================] - 20s 91ms/step - loss: 0.0349 - last_time_step_mse: 0.0169 - val_loss: 0.0314 - val_last_time_step_mse: 0.0132
Epoch 4/20
219/219 [==============================] - 20s 92ms/step - loss: 0.0313 - last_time_step_mse: 0.0140 - val_loss: 0.0290 - val_last_time_step_mse: 0.0120
Epoch 5/20
219/219 [==============================] - 20s 89ms/step - loss: 0.0286 - last_time_step_mse: 0.0117 - val_loss: 0.0280 - val_last_time_step_mse: 0.0104
Epoch 6/20
219/219 [==============================] - 20s 91ms/step - loss: 0.0279 - last_time_step_mse: 0.0115 - val_loss: 0.0255 - val_last_time_step_mse: 0.0086
Epoch 7/20
219/2

In [ ]:
#deep rnn with layer normalizatio

#we have two problems with rnns
#unstable gradients and short memory
#to deal with gradients we can use most techniques we know (proper initialization, faster optimizers, dropout etc. batch normalization doesn't work well)
#we have one more new tool - layer normalization
# instead of normalizing across a batch it normalizes across features


from tensorflow.keras.layers import LayerNormalization

#this will do  exactly the same as SimpleRNNCell with also Layer Norm at each time step
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [ ]:

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20, verbose =3,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [ ]:
#my first LSTM
model = keras.models.Sequential([
                                 keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.LSTM(20, return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])



model.compile(loss="mse", optimizer="adam" , metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20, verbose =2,

                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 - 5s - loss: 0.1667 - last_time_step_mse: 0.1497 - val_loss: 0.1011 - val_last_time_step_mse: 0.0694
Epoch 2/20
219/219 - 4s - loss: 0.0792 - last_time_step_mse: 0.0433 - val_loss: 0.0649 - val_last_time_step_mse: 0.0274
Epoch 3/20
219/219 - 4s - loss: 0.0580 - last_time_step_mse: 0.0210 - val_loss: 0.0526 - val_last_time_step_mse: 0.0168
Epoch 4/20
219/219 - 4s - loss: 0.0485 - last_time_step_mse: 0.0141 - val_loss: 0.0452 - val_last_time_step_mse: 0.0127
Epoch 5/20
219/219 - 4s - loss: 0.0429 - last_time_step_mse: 0.0108 - val_loss: 0.0411 - val_last_time_step_mse: 0.0105
Epoch 6/20
219/219 - 4s - loss: 0.0395 - last_time_step_mse: 0.0095 - val_loss: 0.0383 - val_last_time_step_mse: 0.0089
Epoch 7/20
219/219 - 4s - loss: 0.0372 - last_time_step_mse: 0.0088 - val_loss: 0.0359 - val_last_time_step_mse: 0.0080
Epoch 8/20
219/219 - 4s - loss: 0.0353 - last_time_step_mse: 0.0081 - val_loss: 0.0344 - val_last_time_step_mse: 0.0073
Epoch 9/20
219/219 - 4s - loss: 0.0339 -

In [ ]:

np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])


np.random.seed(42)

tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, Y_valid[:, 3::2]))

Epoch 1/20
219/219 [==============================] - 2s 9ms/step - loss: 0.1388 - last_time_step_mse: 0.1302 - val_loss: 0.0827 - val_last_time_step_mse: 0.0710
Epoch 2/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0696 - last_time_step_mse: 0.0553 - val_loss: 0.0553 - val_last_time_step_mse: 0.0402
Epoch 3/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0474 - last_time_step_mse: 0.0308 - val_loss: 0.0403 - val_last_time_step_mse: 0.0245
Epoch 4/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0360 - last_time_step_mse: 0.0196 - val_loss: 0.0322 - val_last_time_step_mse: 0.0168
Epoch 5/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0296 - last_time_step_mse: 0.0140 - val_loss: 0.0274 - val_last_time_step_mse: 0.0122
Epoch 6/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0257 - last_time_step_mse: 0.0106 - val_loss: 0.0244 - val_last_time_step_mse: 0.0099
Epoch 7/20
219/219 [========

In [ ]:
#wavenet

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                  activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1)) # final layer to predict 10 next values
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1176 - last_time_step_mse: 0.0984 - val_loss: 0.0543 - val_last_time_step_mse: 0.0334
Epoch 2/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0472 - last_time_step_mse: 0.0261 - val_loss: 0.0415 - val_last_time_step_mse: 0.0220
Epoch 3/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0391 - last_time_step_mse: 0.0194 - val_loss: 0.0367 - val_last_time_step_mse: 0.0180
Epoch 4/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0351 - last_time_step_mse: 0.0159 - val_loss: 0.0332 - val_last_time_step_mse: 0.0148
Epoch 5/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0323 - last_time_step_mse: 0.0134 - val_loss: 0.0316 - val_last_time_step_mse: 0.0134
Epoch 6/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0305 - last_time_step_mse: 0.0120 - val_loss: 0.0297 - val_last_time_step_mse: 0.0120
Epoch 7/20
219/219 [========